## Assignment 4

In [19]:
import pandas as pd

In [27]:
df = pd.read_csv("C:/Users/PRIYANKA/OneDrive - Syracuse University/Text Mining/Assignment 4/pubmed_causal_language_use_1.csv")

In [28]:
df.head()

,sentence,label
0,Levels of cholesterol fractions in patients wi...,0
1,Faster aspart and IAsp were confirmed noninfer...,0
2,Major operative morbidity after minimally inva...,0
3,"The promise of combining risk assessment, comm...",0
4,PPARÎ´ peroxisome proliferator-activated recep...,0


In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

NameError: name 'X' is not defined

In [10]:
# prepare train and test data
# since fine tuning a BERT model still requires a significant amount of time, 
# only 1000 training examples and 1000 test examples will be used for demo purpose.
# prior experiment shows LinearSVC's best accuracy (3-fold CV) is about 62-65% depending on vectorization options
# BERT should be able to outperform LinearSVC with far fewer training examples.

dff = df.sample(frac=1)
#train_size = int(0.01 * len(dff))
#df_train = dff[:train_size]
#df_test = dff[train_size:]
df_train = dff[:1000]
df_test = dff[-1000:]
print(df_train.shape)
print(df_test.shape)
X_train, y_train = df_train['sentence'].values, df_train['label'].values
X_test, y_test = df_test['sentence'].values, df_test['label'].values

(1000, 2)
(1000, 2)


In [11]:
# check the category distribution in the train and test set

import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)))

[[  0   1   2   3]
 [453 142  71 334]]
[[  0   1   2   3]
 [450 173  67 310]]


In [12]:
# build a LinearSVC model as a baseline comparison to the BERT model
# since LinearSVC is a linear model, we can print out its top features in each category to see whether the model learned something meaningful
# in this example, the top 10 features for the "very negative" category (category 0) is printed out
# you can see that some top features are not so negative
# with only 1000 training examples, LinearSVC achieved 0.529 accuracy on the 1000 test examples.

from sklearn.feature_extraction.text import CountVectorizer
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1, max_iter=2000)
svm_clf.fit(X_train_vec,y_train)
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
very_negative_10 = feature_ranks[-10:]
print("Very negative words")
for i in range(0, len(very_negative_10)):
    print(very_negative_10[i])
print()


Very negative words
(0.6490927754614901, 'missing')
(0.6499411273917953, 'concentrations')
(0.6712995425024264, 'understand')
(0.67930712523442, 'arterial')
(0.7117815670678607, 'noninferior')
(0.724891578151237, 'studies')
(0.8362230712675192, 'drugs')
(0.888998094910451, 'considered')
(0.9606683617871902, 'despite')
(1.0309617053668034, 'needed')



In [13]:
# to save some time, we are using a hold-out test to compare the LinearSVC and BERT models
# you can also try cross validation
# LinearSVC test accuracy
X_test_vec = unigram_count_vectorizer.transform(X_test)
y_pred = svm_clf.predict(X_test_vec)
svm_clf.score(X_test_vec,y_test)

# you can also try cross validation on the train set
# the result should not be far from the hold-out test
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
svm_pipe = Pipeline([('vect', unigram_count_vectorizer),('svm',LinearSVC(dual=True, max_iter=2000))])
scores = cross_val_score(svm_pipe,X_train,y_train,cv=3)
print(sum(scores)/len(scores))

0.6569833306360252


In [17]:
# print confusion matrix and classification report

from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3])
print(cm)
print()

from sklearn.metrics import classification_report
target_names = ['0','1','2','3']
print(classification_report(y_test, y_pred, target_names=target_names))

# LinearSVC error analysis
# print out errors to check if any common patterns for further model improvement

# print out very positive examples that were predicted as negative
# you can print out different types of errors 
err_cnt = 0
for i in range(0, len(y_test)):
  if (y_test[i]==3 and y_pred[i]==0):
    print(X_test[i])
    err_cnt = err_cnt+1
print("errors:", err_cnt)

[[352  25  18  55]
 [ 64  65  14  30]
 [ 25  12  17  13]
 [ 68  13   5 224]]

              precision    recall  f1-score   support

           0       0.69      0.78      0.73       450
           1       0.57      0.38      0.45       173
           2       0.31      0.25      0.28        67
           3       0.70      0.72      0.71       310

    accuracy                           0.66      1000
   macro avg       0.57      0.53      0.54      1000
weighted avg       0.65      0.66      0.65      1000

Few studies have addressed the topic, and the results of these studies provide only limited support for an increased risk of post-operative cognitive dysfunction in patients who are obese.
They tended to be older, socio-economically disadvantaged, and more fragile.
Risk factor screening methods are poor predictors of which pregnant  women will be diagnosed with GDM.
Rates of recurrent preeclampsia among women with a history of preeclampsia decreased by 30% after release of the US Pr